In [1]:
import os
import shutil

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import habitat
from habitat.core.utils import try_cv2_import

from habitat.tasks.nav.shortest_path_follower import ShortestPathFollower
from habitat.utils.visualizations import maps
from habitat.utils.visualizations.utils import images_to_video

from tqdm.notebook import tqdm, trange
import seaborn as sns
sns.set(style='dark')

from PIL import Image
from habitat_sim.utils.common import d3_40_colors_rgb

actions_number = 5

In [2]:
cv2 = try_cv2_import()

class SimpleRLEnv(habitat.RLEnv):
    def get_reward_range(self):
        return [-1, 1]

    def get_reward(self, observations):
        return 0

    def get_done(self, observations):
        return self.habitat_env.episode_over

    def get_info(self, observations):
        return self.habitat_env.get_metrics()

In [3]:
def draw_top_down_map(info, heading, output_shape):
    output_size = output_shape[1]
    top_down_map = maps.colorize_topdown_map(info["top_down_map"]["map"], 
                                             info["top_down_map"]["fog_of_war_mask"])
    original_map_size = top_down_map.shape[:2]
    map_scale = np.array((1, original_map_size[1] * 1.0 / original_map_size[0]))
    new_map_size = np.round(output_size * map_scale).astype(np.int32)
    # OpenCV expects w, h but map size is in h, w
    top_down_map = cv2.resize(top_down_map, (new_map_size[1], new_map_size[0]))

    map_agent_pos = info["top_down_map"]["agent_map_coord"]
    map_agent_pos = np.round(
        map_agent_pos * new_map_size / original_map_size
    ).astype(np.int32)
    top_down_map = maps.draw_agent(
        top_down_map,
        map_agent_pos,
        heading - np.pi / 2,
        agent_radius_px=top_down_map.shape[0] / 40,
    )
    return cv2.resize(top_down_map, (output_shape[1], output_shape[0]))

## Init config and Run environment

In [4]:
config = habitat.get_config(config_paths='my_challenge_objectnav2020.local.rgbd.yaml')
# config = habitat.get_config(config_paths='my_challenge_pointnav2020.local.rgbd.yaml')
config.defrost()
# config.DATASET.SPLIT = "train"
width = height = 256
config.SIMULATOR.RGB_SENSOR.WIDTH = width
config.SIMULATOR.RGB_SENSOR.HEIGHT = height
config.SIMULATOR.DEPTH_SENSOR.WIDTH = width
config.SIMULATOR.DEPTH_SENSOR.HEIGHT = height
config.SIMULATOR.SEMANTIC_SENSOR.WIDTH = width
config.SIMULATOR.SEMANTIC_SENSOR.HEIGHT = height
config.freeze()

In [5]:
from habitat.datasets.object_nav.object_nav_dataset import ObjectNavDatasetV1
dataset = ObjectNavDatasetV1(config.DATASET)

env = SimpleRLEnv(config=config, dataset=dataset)
# Forces env to switch scenes
env._env.episode_iterator = dataset.get_episode_iterator(max_scene_repeat_episodes=1)

goal_radius = env.episodes[0].goals[0].radius
if goal_radius is None:
    goal_radius = config.SIMULATOR.FORWARD_STEP_SIZE

follower = ShortestPathFollower(env.habitat_env.sim, goal_radius, False)
mode = "approximate_gradient"
follower.mode = mode

2020-08-10 13:39:58,862 initializing sim Sim-v0
initializing sim Sim-v0
2020-08-10 13:40:05,804 Initializing task ObjectNav-v1
Initializing task ObjectNav-v1


In [6]:
set(ep.scene_id for ep in env._env._dataset.episodes)

{'/data/scene_datasets/mp3d/x8F5xyUWy9e/x8F5xyUWy9e.glb'}

## Make steps

In [8]:
from collections import defaultdict
log_images = defaultdict(lambda: [])

for j in range(54):
    observations = env.reset()
    print('playing', env.habitat_env.current_episode.scene_id)
    im = observations["rgb"]
    done = False

    for i in trange(500):
        if not done:
            best_action = follower.get_next_action(env.habitat_env.current_episode.goals[0].position)
            if best_action is None:
                best_action = 0

            observations, reward, done, info = env.step(best_action)
            im = observations["rgb"]
            top_down_map = draw_top_down_map(info, 
                                             observations["heading"][0], 
                                             im.shape)
            output_im = np.concatenate((im, top_down_map), axis=1)
            output_resized = cv2.resize(np.rollaxis(output_im, 0, 1), (im.shape[0]*2, im.shape[0]))
            log_images['debug'].append(output_resized)
            log_images['rgb'].append(observations['rgb'])
            log_images['depth'].append(observations['depth'])
            log_images['semantic'].append(observations['semantic'])
        else:
            print('Episode is done!')
            break

playing /data/scene_datasets/mp3d/x8F5xyUWy9e/x8F5xyUWy9e.glb


Episode is done!

playing /data/scene_datasets/mp3d/x8F5xyUWy9e/x8F5xyUWy9e.glb


KeyboardInterrupt: 

In [9]:
from moviepy.editor import ImageSequenceClip
ImageSequenceClip(log_images['debug'], fps=20).ipython_display(maxduration=1000)

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4

Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [ ]:
from depth_utils import get_point_cloud_from_z, get_camera_matrix, bin_points

In [ ]:
width = config.SIMULATOR.DEPTH_SENSOR.WIDTH
height = config.SIMULATOR.DEPTH_SENSOR.HEIGHT
fov = config.SIMULATOR.DEPTH_SENSOR.HFOV
camera_matrix = get_camera_matrix(width, height, fov)
point_cloud = get_point_cloud_from_z(observations['depth'][..., 0], camera_matrix)

In [ ]:
def semantic_to_rgb(semantic_obs):
    semantic_img = Image.new("P", (semantic_obs.shape[1], semantic_obs.shape[0]))
    semantic_img.putpalette(d3_40_colors_rgb.flatten())
    semantic_img.putdata((semantic_obs.flatten() % 40).astype(np.uint8))
    semantic_img = semantic_img.convert("RGBA")
    return np.array(semantic_img)

semantic = semantic_to_rgb(observations['semantic'])

In [ ]:
observations['semantic'].shape

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(im)
plt.subplot(1, 2, 2)
plt.imshow(semantic)

In [ ]:
plt.figure(figsize=(10, 5))
point_cloud_flat = point_cloud.reshape((-1, 3))
colors = semantic.reshape((-1, 4))
ax = plt.axes(projection='3d')
ax.scatter(point_cloud_flat[:,0], point_cloud_flat[:,1], point_cloud_flat[:,2], c=colors/255, s=0.01)

In [ ]:
%matplotlib agg
log_images['3d'] = []
for i in trange(len(log_images['depth'])):
    fig = plt.figure(figsize=(7, 3))

    point_cloud = get_point_cloud_from_z(log_images['depth'][i][..., 0], camera_matrix)    
    point_cloud_flat = point_cloud.reshape((-1, 3))
    semantic = semantic_to_rgb(log_images['semantic'][i])
    colors = semantic.reshape((-1, 4))
    ax = plt.axes(projection='3d')
    ax.scatter(point_cloud_flat[:,0], point_cloud_flat[:,1], point_cloud_flat[:,2], c=colors/255, s=0.01)
    fig.tight_layout()
    fig.canvas.draw()

    # Now we can save it to a numpy array.
    data = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
    data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    log_images['3d'].append(data)

In [ ]:
ImageSequenceClip(log_images['3d'], fps=10).ipython_display(maxduration=1000)

In [ ]:
images = []
for i in trange(len(log_images['depth'])):
    base = np.zeros((600, 700, 3)).astype(np.uint8)
    base[0:300, 0:700] = log_images['3d'][i]
    base[300:600, 0:300] = cv2.resize(log_images['rgb'][i], (300,300))
    semantic = semantic_to_rgb(log_images['semantic'][i])[...,:3]
    base[300:600, 300:600] = cv2.resize(semantic, (300,300))
    images.append(base)

In [ ]:
ImageSequenceClip(images, fps=10).ipython_display(maxduration=1000)

In [ ]:
%matplotlib inline

plt.figure(figsize=(5, 5))
plt.imshow(data)

In [ ]:
plt.imshow(log_images['depth'][0][...,0])

In [ ]:
bin_points(point_cloud, 200, [0.5, 6], 1).shape

In [ ]:
bins = bin_points(point_cloud, 200, [0.0, 3], 1)
plt.imshow(bins[...,1]*10)

In [ ]:
from IPython.display import Video
import moviepy.video.io.ImageSequenceClip
folder_path = "../Neural-SLAM/tmp/dump/exp1/episodes/1/1/"
image_files = os.listdir(folder_path)
image_files = sorted(os.listdir(folder_path), key=lambda x: int(x.split('-')[-1].split('.')[0]))
image_files = [os.path.join(folder_path, file) for file in image_files]
clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=20)

In [ ]:
a = clip.write_videofile('tmp.mp4')

In [ ]:
Video('tmp.mp4')

In [ ]:
env.habitat_env.current_episode.goals[0].position